In [1]:
import contracts
contracts.disable_all()
import duckietown_world as dw
from duckietown_world.svg_drawing.ipython_utils import ipython_draw_html
dw.logger.setLevel(50)

# get_lane_poses(dw: PlacedObject, q: SE2v, tol=0.000001) -> Iterator[GetLanePoseResult]:



DEBUG:commons:version: 6.1.7 *
INFO:typing:version: 6.1.8
DEBUG:duckietown_world:duckietown-world version 6.2.18 path /home/ale/gitclone/duckietown-world/src
INFO:geometry:version: 2.0.5
DEBUG:aido_schemas:aido-protocols version 6.0.41 path /home/ale/gitclone/duckietown-world/venv/lib/python3.8/site-packages
INFO:nodes:version 6.1.1 path /home/ale/gitclone/duckietown-world/venv/lib/python3.8/site-packages pyparsing 2.4.7


First, we load a map.

In [2]:
m = dw.load_map('loop_obstacles')
ipython_draw_html(m);

INFO:duckietown_world: data: /home/ale/gitclone/duckietown-world/src/duckietown_world/data
INFO:duckietown_world: area: RectangularArea(pmin=[ 0.  -0.1],pmax=[4.44599981 3.86099977])
DEBUG:commons.fs:Written 0.4MB to: out/ipython_draw_html/140022858807952/drawing.html
INFO:duckietown_world:Written SVG
 fn_svg: out/ipython_draw_html/140022858807952/drawing.svg


Now let's sample an initial pose.

In [3]:
from duckietown_world.world_duckietown.sampling_poses import sample_good_starting_pose
q = sample_good_starting_pose(m, only_straight=False, along_lane=0.02)

We use the `get_lane_pose` function to get the list of lanes in which the Duckiebot could be.

In [4]:
from duckietown_world.world_duckietown import get_lane_poses
possibilities = list(get_lane_poses(m, q))


# for i in range(4):
    # m.set_object('db18-%s'% i, dw.DB18(), ground_truth=dw.SE2Transform.from_SE2(q))
# ipython_draw_html(m)

print(f"There are {len(possibilities)} possibilities")
lpr = possibilities[0]
from zuper_typing import debug_print
print(debug_print(lpr)
      )

There are 1 possibilities
GetLanePoseResult
│ tile:
│ Tile
│ │ children:
│ │ dict[1]
│ │ │ curve_left:
│ │ │ PlacedObject
│ │ │ │ children:
│ │ │ │ dict[1]
│ │ │ │ │ curve:
│ │ │ │ │ PlacedObject
│ │ │ │ │ │ children: {lane1: LaneSegment(), lane2: LaneSegment()}
│ │ │ │ │ │ spatial_relations:
│ │ │ │ │ │ dict[2]
│ │ │ │ │ │ │ lane1:
│ │ │ │ │ │ │ GroundTruth
│ │ │ │ │ │ │ │ a: ()
│ │ │ │ │ │ │ │ transform: SE2Transform([0.0, 0.0],0.0)
│ │ │ │ │ │ │ │ b: (lane1,)
│ │ │ │ │ │ │ lane2:
│ │ │ │ │ │ │ GroundTruth
│ │ │ │ │ │ │ │ a: ()
│ │ │ │ │ │ │ │ transform: SE2Transform([0.0, 0.0],90.0)
│ │ │ │ │ │ │ │ b: (lane2,)
│ │ │ │ spatial_relations:
│ │ │ │ dict[1]
│ │ │ │ │ curve:
│ │ │ │ │ GroundTruth
│ │ │ │ │ │ a: ()
│ │ │ │ │ │ transform: SE2Transform([0.0, 0.0],270.0)
│ │ │ │ │ │ b: (curve,)
│ │ spatial_relations:
│ │ dict[1]
│ │ │ 1:
│ │ │ GroundTruth
│ │ │ │ a: ()
│ │ │ │ transform: SE2Transform([0.0, 0.0],0.0)
│ │ │ │ b: (curve_left,)
│ tile_fqn: (tilemap, tile-4-1)
│ tile_transform: Tr